# Question 4: General Theory/Application

_No need to be verbose, it's not fun for anyone_

1. What part of S**L**A**M** did this project deal with? Why? What does the other part deal with and how would it generally work, given that you only have LIDAR scans, RGB video stream, and noisy pose data for a moving robot?


2. Loop closures play an important role in reducing drift, how would you go about detecting these?


3. Explain the structure of your Jacobian. Is the pose-graph fully connected? Why/Why not?


4. With what you know now, how would you describe and differentiate the SLAM frontend and backend? Why do we need to optimise our poses/map in the first place - where does the noise come from/why?

1. The project dealt with the backend SLAM problem, it specifically focused on the localization part. This is evident from the information provided to us, namely, odometry details, loop closure constraints, and the anchor node and from what was expected of us, i.e., determining the optimal graph given a particular information matrix, aka localization. The project did not involve anything related to mapping of the robot’s environment, which is what the other part of SLAM deals with.
For mapping, we try to obtain highly informative and distinguishable landmarks. These landmarks can be obtained from our RGB video stream based on some threshold image energy. More formally, the landmarks are known as the SURF features. These features not only help us in localization but also make mapping the rest of the environment much easier. In order to do that we need to determine the camera pose, an addition of an IMU sensor makes our jobs easier and determines the roll and pitch for us. Therefore, the 3D positions of two image features are now sufficient to determine the camera pose. To add the new image features to the map, the already observed image features need to be tracked which can be done by matching the rgb values, using GPS, or by using kd trees. For matching these image features need to be taken back to the global frame, which is done by the odometry received through our localization (Lidar scans). So, we clearly see how localization and mapping are dependent on each other. A good localisation creates a better mapping of the environment and a good mapping creates very informative loop closure constraints which helps the robot to localize itself.  
It is also interesting to note that the mapping problem can be reduced to almost nothing, by making extensive use of highly detailed map data collected in advance. For example, Google's StreetView may be used as part of maps. Essentially such systems simplify the SLAM problem to a simpler localization only task, perhaps allowing for moving objects such as cars and people only to be updated in the map at runtime.

2. Loop closures form the backbone of the backend SLAM. It is through them that we actually get to correct our inaccurate graph. 
Because of noise, we do not usually get the correct pose, so in practical situations the robot may reach the same spot as it did before, but estimate itself at some other place. This leads to a very poor graph as at each frame we keep adding our error of wrong estimation. To deal with this, we try to find which poses of the estimated graph are actually the same points, and based on that using Localisation methods, we optimise the graph(as we did in this assignment). But in the assignment we were provided with the loop closure information beforehand, so how are they calculated?
So first we need to use some kind of sensor like LiDAR,gps or stereo camera. Using that we keep storing their data for each pose. Now we need to relate/match this data between different poses to make a loop. We can do that using:  


    1.Iterative Closest Point -SLAM: This is implemented by first sampling some poses based on current pose (usually by taking poses in some close range). Now in these sample points, apply ICP so that we can align the current observation with the observations in the samples. And now select the match based on some threshold. 
    Problems with the approach:   
        a) ICP is dependent on initial guess.  
        b) Inefficient Sampling.  
        c) Dependence on environment - Let's say we are in a room. As we use visual data as seen in front of our robot, the robot may see two different corners of the room and put them in the same sample, which would lead to incorrect loops.  
    ICP is also known as map-to-map matching, as it finds correspondences b/w common features.  
    2. Image-to-map : This method uses relocalisation technique used to recover from tracking failure to detect a loop. It does so by determining the pose of the camera relative to map of point features by finding correspondences b/w them. After this, RANSAC is used to determine pose from the correspondences.  
    3. GPS-SLAM: This method involves using GPS as the primary sensor.
    
    One of the most important things regarding Loop Closure is that until we are not very sure regarding a loop edge, it is better not to add it! This is to avoid the wrong information it can bring with it, and it will impact our estimation more badly than it would do any good.

3. The structure of Jacobian need not be unique, it is determined by the order of functions and variables chosen. In our jacobian, the function order starts with that of the anchor edge, then comes the odometry edges followed by the loop closure edges. For variables the order is similar to that of the nodes, i.e., x0, y0, z0, x1, y1, z1, and so on. starts with all the x coordinates then y and z. Therefore, the jacobian has 3*(O + L + 1) rows and 3*V columns where O, L, and V represent odometry edges, loop closure edges, and the number of nodes respectively. The cell (i, j) in the jacobian represents the differentiation of the ith function with the jth variable. More formally, each column of Jacobian is the partial derivative of residual with respect to frame. It does so by computing partial derivatives w.r.t. x_i,y_i and theta_i and then stretching them as three columns. A rough sketch of the Jacobian is given below.

![Theory](../misc/Q3-images/jacobian.jpeg "Jacobian")  

If by connected the question means each node is reachable from any node (by transformations), then the graph would always be connected when any two nodes have any edge connecting them irrespective of the loop closures. This is because if we have odometry information between any two consecutive poses we can always use inverse transformations to go in the opposite direction.    
If connected means that there would always be an edge connecting any two nodes then the graph would in most cases not be fully connected. It is only possible when the loop closures connect all the nodes to each other. Moreover, the sparsity of the Jacobian matrix and the summation of the H matrix both give us a good idea about how the edges are distributed over the graph. It is also interesting to note that when the environment becomes larger, the adjacency matrix becomes sparser, which actually makes sense.


4. The overall slam has two components to it, namely the frontend and the backend. The frontend deals with the construction of the graph which involves making anchor, odometry, and loop closure edges. The backend deals with graph optimization, which is what we did for the assignment, geometrically it means that we tighten the stronger edges which changes the nodes locations but not the edges length so as to make the graph look more sensible.   
But the problem lies in the Frontend, the odometry edges are not what they should be. These edges are obtained by applying the iterative closest point algorithm on the lidar scans or from the wheel odometry from the two consecutive nodes. The former are highly sensitive to initial correspondances and accuracy in the measurements of the data points, hence are very inaccurate. The noisy lidar depths are very problematic, this noise arises from the high sensitivity of the detector down to the single-photon level, the point clouds very often show a lot of points in the air, moreover, taking lidar scans while in motion induces velocity bound erros. Low vertical accuracy relative to horizontal sample distance also gives rise to noise. The wheel odometry takes into account the number of rotations travelled from the initial node to the next. This suffers from the slips and drifts which creates significant errors in the odometry edges. No matter which method we use, these small inaccuracies get accumulated successively through each edge which sums up to a highly inaccurate overall graph, making way for the backend to solve this problem.